In [65]:
# imports
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [145]:
# Load bike station data from previously exported .csv (See citybikes.ipynb)
station_data = pd.read_csv("../data/citybike_stations.csv")

station_data

,station_id,station_name,free_bikes,empty_slots,latitude,longitude
0,af1d0f25cbc75377878349fde4d86133,Merchant Square - ELECTRIC,7,2.0,55.858167,-4.245483
1,71b02b6f26f03e58c97dec1e1c9faa00,Botanic Gardens - ELECTRIC,1,7.0,55.878278,-4.288487
2,fa6bc682e8627d66891c2b6250c81b1e,Mitchell Library,7,0.0,55.864788,-4.271852
3,a087989280fd941fd9c41c3e787940ae,University of Strathclyde (North),1,2.0,55.862983,-4.241000
4,eb47a36e3d5fe5bc456397413bd2aa75,Bridge Street Subway - ELECTRIC,5,3.0,55.852500,-4.258850
...,...,...,...,...,...,...
102,2c49302c8bb3a00373f898594228a941,London Road Central,1,9.0,55.847638,-4.202957
103,d9eacbf78518d186400386d90f0cf49c,Broomhill Cross,14,0.0,55.875865,-4.319601
104,f6210447e1724487afe0ce63b011aecd,Knightswood Park,5,5.0,55.895974,-4.354877
105,3860eb7809fcced150d10d78ca2a165c,"University Hospital, Queen Eliza",8,0.0,55.862838,-4.341885


In [146]:
# Create a dictionary containing the station id (for later use), and formatted coordinates to be used in the below API call
station_coordinates = []

# For every item & row in the CSV
for index, row in station_data.iterrows():
    # Gather the id, lat & long for each station
    station_id = row['station_id']
    lat = row['latitude']
    lon = row['longitude']
    # Load the values into a dictionary
    station_data = {'station_id': station_id, 'coords': f"{lat},{lon}"}
    # Add it to our coordinate list
    station_coordinates.append(station_data)

# Headers & parameters needed for FourSquare
headers = {"Accept": "application/json"}
headers['Authorization'] = os.environ["FOURSQUARE_API_KEY"]
radius_to_search = 1000 # Distance of POI from bike station location (in meters)

# Send an API request for every pair of station coordinates to compile POI data
poi_data = []
for station in station_coordinates:
    # Create the API call using the bike station location and provided search radius
    request_url = "https://api.foursquare.com/v3/places/search?ll=" + station['coords'] + "&radius=" + str(radius_to_search)
    response = requests.get(request_url, headers=headers)
    poi_data.append({'station_id': station['station_id'], 'data': response.json()}) # Return the station ID and JSON response

Load the API data into a DataFrame

In [162]:
# NOTE to self: Run this code block to avoid repeated API calls

# Load the compiled data into Pandas dataframe 
poi_data_df = pd.DataFrame(poi_data)

poi_data_df

,station_id,data
0,af1d0f25cbc75377878349fde4d86133,{'results': [{'fsq_id': '54c7cade498e300defb09...
1,71b02b6f26f03e58c97dec1e1c9faa00,{'results': [{'fsq_id': '4b9bc057f964a5201c213...
2,fa6bc682e8627d66891c2b6250c81b1e,{'results': [{'fsq_id': '4b852d7bf964a5207d4f3...
3,a087989280fd941fd9c41c3e787940ae,{'results': [{'fsq_id': '57c45369498e5276e3a65...
4,eb47a36e3d5fe5bc456397413bd2aa75,{'results': [{'fsq_id': '4b1583cef964a520fbad2...
...,...,...
102,2c49302c8bb3a00373f898594228a941,{'results': [{'fsq_id': '4b11505bf964a520237a2...
103,d9eacbf78518d186400386d90f0cf49c,{'results': [{'fsq_id': '55214a32498e62c5cbccb...
104,f6210447e1724487afe0ce63b011aecd,{'results': [{'fsq_id': '4c4f44a01d76c928f1a19...
105,3860eb7809fcced150d10d78ca2a165c,{'results': [{'fsq_id': '59fa2484f62e093f9af7a...


In [163]:
results_df_list = [] # Empty list to store our results

# Iterate through every row in the POI dataframe
for index, row in poi_data_df.iterrows():
    # Extract the foursquare results from the data column
    results = row['data']['results']
    
    # Check if the data exists (without this, excess blank rows will be created)
    if len(results) > 0:
        # Populate a dataframe with the extracted data
        results_df = pd.DataFrame(results)
        # Extract the station ID from the Citybikes API
        results_df['station_id'] = row['station_id']

        results_df_list.append(results_df)

# Combine the results from this list across the rows (axis 0) back into our dataframe
poi_data_df = pd.concat(results_df_list, axis=0)

poi_data_df

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone,station_id
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London,af1d0f25cbc75377878349fde4d86133
1,4bc35be22a89ef3b08e7f488,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...",[],34,"{'main': {'latitude': 55.858078, 'longitude': ...",/v3/places/4bc35be22a89ef3b08e7f488,"{'address': '44 Candleriggs', 'admin_region': ...",The Dhabba,{},Europe/London,af1d0f25cbc75377878349fde4d86133
2,4b9cecbcf964a520bd8236e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],43,"{'main': {'latitude': 55.858429, 'longitude': ...",/v3/places/4b9cecbcf964a520bd8236e3,"{'address': '71-73 Albion St', 'admin_region':...",Merchant Square,{'children': [{'fsq_id': '595f841bf79faa623f93...,Europe/London,af1d0f25cbc75377878349fde4d86133
3,4ba0e8c3f964a520508637e3,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...",[],98,"{'main': {'latitude': 55.85886, 'longitude': -...",/v3/places/4ba0e8c3f964a520508637e3,"{'admin_region': 'Scotland', 'country': 'GB', ...",City Halls,{},Europe/London,af1d0f25cbc75377878349fde4d86133
4,554cdd3c498ee17d5a870117,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[],144,"{'main': {'latitude': 55.859412, 'longitude': ...",/v3/places/554cdd3c498ee17d5a870117,"{'address': '127 Candleriggs', 'admin_region':...",Spitfire Espresso,{},Europe/London,af1d0f25cbc75377878349fde4d86133
...,...,...,...,...,...,...,...,...,...,...,...
5,06b2e83c632e418806153b09,"[{'id': 17010, 'name': 'Used Car Dealership', ...",[],421,"{'main': {'latitude': 55.845601, 'longitude': ...",/v3/places/06b2e83c632e418806153b09,"{'address': '136 Boden St', 'admin_region': 'S...",London Cab Sales,{},Europe/London,ed8440d874a0b4205ddbcc0bc56f3d49
6,c6e7c00cef054a0a92e80063,"[{'id': 16037, 'name': 'Playground', 'icon': {...",[],543,"{'main': {'latitude': 55.842511, 'longitude': ...",/v3/places/c6e7c00cef054a0a92e80063,"{'address': '421 Baltic St', 'admin_region': '...",Baltic Street Adventure Playground,{},Europe/London,ed8440d874a0b4205ddbcc0bc56f3d49
7,4c1ae6aeb9f876b044847946,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],557,"{'main': {'latitude': 55.848546, 'longitude': ...",/v3/places/4c1ae6aeb9f876b044847946,"{'address': '851 London Rd', 'admin_region': '...",London Road Police Station,{},Europe/London,ed8440d874a0b4205ddbcc0bc56f3d49
8,5bc4936d898bdc002bfa5d46,"[{'id': 15003, 'name': 'Alternative Medicine C...",[],666,"{'drop_off': {'latitude': 55.840302, 'longitud...",/v3/places/5bc4936d898bdc002bfa5d46,"{'address': '45 Auckland Wynd', 'admin_region'...",Certified Professional Never Binge Again Maste...,{},Europe/London,ed8440d874a0b4205ddbcc0bc56f3d49


In [164]:
# Renaming some columns to make the contents more obvious
poi_data_df.rename(columns={'distance':'distance_from_station'}, inplace=True)
poi_data_df.rename(columns={'name':'poi_name'}, inplace=True)

# Display altered column headers
poi_data_df.head(1)

,fsq_id,categories,chains,distance_from_station,geocodes,link,location,poi_name,related_places,timezone,station_id
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London,af1d0f25cbc75377878349fde4d86133


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [165]:
# Copy the desired columns into the final dataframe
selected_poi_data_df = poi_data_df[['station_id','distance_from_station','poi_name','categories','location']].copy()

# Display results
selected_poi_data_df

,station_id,distance_from_station,poi_name,categories,location
0,af1d0f25cbc75377878349fde4d86133,31,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc..."
1,af1d0f25cbc75377878349fde4d86133,34,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ..."
2,af1d0f25cbc75377878349fde4d86133,43,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':..."
3,af1d0f25cbc75377878349fde4d86133,98,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ..."
4,af1d0f25cbc75377878349fde4d86133,144,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':..."
...,...,...,...,...,...
5,ed8440d874a0b4205ddbcc0bc56f3d49,421,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S..."
6,ed8440d874a0b4205ddbcc0bc56f3d49,543,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '..."
7,ed8440d874a0b4205ddbcc0bc56f3d49,557,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '..."
8,ed8440d874a0b4205ddbcc0bc56f3d49,666,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'..."


Extract the important values from the Category & Location columns

In [166]:
# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a tuple
    # NOTE: Tried list before, but .drop_duplicates() does not work on that data type so I had to settle with tuples
    category_tuple = ()
    for category in categories_dic:
        category_tuple += (category['name'],) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_tuple

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address', '')
    
    return poi_address

# Use pandas .apply() dataframe function to extract the important values into new columns for much easier analysis
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
selected_poi_data_df['category_names'] = selected_poi_data_df.apply(get_category_names, axis=1)
selected_poi_data_df['address'] = selected_poi_data_df.apply(get_address, axis=1)

# Display the resulting DataFrame
selected_poi_data_df

,station_id,distance_from_station,poi_name,categories,location,category_names,address
0,af1d0f25cbc75377878349fde4d86133,31,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc...","(Café, Restaurant)",6 Wilson St
1,af1d0f25cbc75377878349fde4d86133,34,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ...","(Indian Restaurant,)",44 Candleriggs
2,af1d0f25cbc75377878349fde4d86133,43,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':...","(Bar, Restaurant)",71-73 Albion St
3,af1d0f25cbc75377878349fde4d86133,98,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ...","(Concert Hall, Theater)",
4,af1d0f25cbc75377878349fde4d86133,144,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':...","(Coffee Shop, Restaurant)",127 Candleriggs
...,...,...,...,...,...,...,...
5,ed8440d874a0b4205ddbcc0bc56f3d49,421,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S...","(Used Car Dealership,)",136 Boden St
6,ed8440d874a0b4205ddbcc0bc56f3d49,543,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '...","(Playground,)",421 Baltic St
7,ed8440d874a0b4205ddbcc0bc56f3d49,557,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '...","(Police Station,)",851 London Rd
8,ed8440d874a0b4205ddbcc0bc56f3d49,666,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'...","(Alternative Medicine Clinic,)",45 Auckland Wynd


Drop the columns that are no longer needed & remove duplicates

In [167]:
# Drop columns
selected_poi_data_df = selected_poi_data_df.drop(['categories','location'], axis=1) # - Commented out as I've already run before and keep getting 'doesn't exist' errors

# Remove duplicates (Will happen in cases when two bike stations are within the search radius of eachother)
# NOTE: Stopped working once I added in the station_id as the rows were no longer exact dupes, had to make function below
#selected_poi_data_df = selected_poi_data_df.drop_duplicates()

# Remove rows with duplicate names & addresses, keeping the value with the shortest distance
# ie. Only keep the shortest path per one venue, otherwise analysis will be inaccurate
shortest_distance_rows = {}
# For every row in the dataframe
for index, row in selected_poi_data_df.iterrows():
    # Gather the important data from current row
    poi_name = row['poi_name']
    address = row['address']
    distance_from_station = row['distance_from_station']
    
    # Check if we have already seen this combination before (ie duplicate POI)
    if (poi_name, address) in shortest_distance_rows:
        # Is the distance to station shorter than what we have saved?
        if distance_from_station < shortest_distance_rows[(poi_name, address)]['distance_from_station']:
            # If so, update our dictionary
            shortest_distance_rows[(poi_name, address)] = row
    else:
        # Otherwise, add it to our dictionary (unseen POI)
        shortest_distance_rows[(poi_name, address)] = row

# Load the compiled values into our dataframe
selected_poi_data_df = pd.DataFrame(shortest_distance_rows.values())

# View the final dataframe to be exported to .CSV
selected_poi_data_df

,station_id,distance_from_station,poi_name,category_names,address
0,af1d0f25cbc75377878349fde4d86133,31,Wilson Street Pantry,"(Café, Restaurant)",6 Wilson St
1,af1d0f25cbc75377878349fde4d86133,34,The Dhabba,"(Indian Restaurant,)",44 Candleriggs
2,af1d0f25cbc75377878349fde4d86133,43,Merchant Square,"(Bar, Restaurant)",71-73 Albion St
3,af1d0f25cbc75377878349fde4d86133,98,City Halls,"(Concert Hall, Theater)",
4,af1d0f25cbc75377878349fde4d86133,144,Spitfire Espresso,"(Coffee Shop, Restaurant)",127 Candleriggs
...,...,...,...,...,...
6,3860eb7809fcced150d10d78ca2a165c,493,Lidl,"(Grocery Store,)",19 Moss Rd
7,3860eb7809fcced150d10d78ca2a165c,448,Motus Commercials Glasgow,"(Car Dealership,)",131 Bogmoor Rd Govan
8,3860eb7809fcced150d10d78ca2a165c,641,Glasgow Centre For Reproductive Medicine,"(Healthcare Clinic,)",21 Fifty Pitches Way
9,3860eb7809fcced150d10d78ca2a165c,681,J K Electrical,"(Electrician,)","17 Fulbar Rd, Cardonald"


In [168]:
# Export to .csv
selected_poi_data_df.to_csv('../data/Foursquare_POIs.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [26]:
# Headers & parameters needed for Yelp API acording to docs
# https://docs.developer.yelp.com/docs/fusion-intro
yelp_headers = {"accept": "application/json"}
yelp_headers['Authorization'] = "Bearer " + os.environ["YELP_API_KEY"]
radius_to_search = 1000

# Example API request url
#yelp_request_url = "https://api.yelp.com/v3/businesses/search?latitude=55.8581672&longitude=-4.245483&radius=1000"

# Load bike station data from previously exported .csv (See citybikes.ipynb)
station_data = pd.read_csv('../data/citybike_stations.csv')

# Format the latitude & longitude of each station for use in our API calls
yelp_POIs = []
#api_limit_counter = 0 

# For each station in the loaded CSV
for index, row in station_data.iterrows():
    #if api_limit_counter >= 2: # Number of times I want to limit the API to run, used for testing
    #    break
    
    # Gather the lat and long for each station
    lat = row['latitude']
    lon = row['longitude']
    yelp_request_url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(lat) + "&longitude=" + str(lon) + "&radius=" + str(radius_to_search)
    response = requests.get(yelp_request_url, headers=yelp_headers)
    yelp_POIs.append(response.json()['businesses'])
    
    #api_limit_counter += 1

# Flatten the results, needed to properly format the list
yelp_POIs = sum(yelp_POIs,[])

Put your parsed results into a DataFrame

In [27]:
# Load the response into a dataframe
# NOTE: Use this cell to reset the DF instead of above to avoid unnessasary API calls
yelp_POIs_df = pd.DataFrame(yelp_POIs)
yelp_POIs_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,JNpi062fXUtd3lmGUSu-ww,café-gandolfi-glasgow-2,Café Gandolfi,https://s3-media3.fl.yelpcdn.com/bphoto/Zh1wQr...,False,https://www.yelp.com/biz/caf%C3%A9-gandolfi-gl...,67,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 55.8580469, 'longitude': -4.2442038}",[],££,"{'address1': '64 Albion Street', 'address2': '...",+441415526813,+44 141 552 6813,85.938424
1,FwZxNZrEYLdvBioEE76QVA,hutchesons-bar-and-brasserie-glasgow,Hutchesons Bar & Brasserie,https://s3-media4.fl.yelpcdn.com/bphoto/NYO7de...,False,https://www.yelp.com/biz/hutchesons-bar-and-br...,51,"[{'alias': 'scottish', 'title': 'Scottish'}, {...",4.0,"{'latitude': 55.859781774815, 'longitude': -4....",[],££,"{'address1': '158 Ingram Street', 'address2': ...",+441415524050,+44 141 552 4050,225.351596
2,ykFuZwPzHrXih4-Y4NAe4Q,brewdog-doghouse-glasgow,BrewDog DogHouse,https://s3-media4.fl.yelpcdn.com/bphoto/0iktlW...,False,https://www.yelp.com/biz/brewdog-doghouse-glas...,48,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,"{'latitude': 55.8593557, 'longitude': -4.2480296}",[],££,"{'address1': '99 Hutcheson Street', 'address2'...",+441415526363,+44 141 552 6363,206.258780
3,9I0QM3wBHiMzX2AvL3OQeg,the-wilson-street-pantry-glasgow,The Wilson Street Pantry,https://s3-media1.fl.yelpcdn.com/bphoto/G7Vz5a...,False,https://www.yelp.com/biz/the-wilson-street-pan...,88,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 55.8585085104567, 'longitude': -4...",[],£,"{'address1': '6 Wilson Street', 'address2': ''...",+441415520606,+44 141 552 0606,48.999836
4,UrnzTa6T3sLSVbbXLiYvQA,paesano-pizza-glasgow,Paesano Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/7KJazO...,False,https://www.yelp.com/biz/paesano-pizza-glasgow...,154,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 55.8596504377382, 'longitude': -4...",[],££,"{'address1': '94 Miller Street', 'address2': '...",+441412585565,+44 141 258 5565,359.689792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,K9O4skNFV5qiiLy5SYprAA,rab-haws-kitchens-glasgow,Rab Haws Kitchens,https://s3-media3.fl.yelpcdn.com/bphoto/IObET9...,False,https://www.yelp.com/biz/rab-haws-kitchens-gla...,2,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]",5.0,"{'latitude': 55.8499589, 'longitude': -4.1965876}",[],NaN,"{'address1': '111 Helenvale Street', 'address2...",+441415508811,+44 141 550 8811,834.579909
1998,nr09_r1igC6tvh-kpov30w,the-bank-glasgow-2,The Bank,,False,https://www.yelp.com/biz/the-bank-glasgow-2?ad...,1,"[{'alias': 'pubs', 'title': 'Pubs'}]",3.0,"{'latitude': 55.8510857, 'longitude': -4.1972446}",[],NaN,"{'address1': '1450 Gallowgate', 'address2': No...",+441415508999,+44 141 550 8999,868.350168
1999,zCU18QVRr0-Pz4M7qBQZ_Q,mcchans-glasgow,McChans,,False,https://www.yelp.com/biz/mcchans-glasgow?adjus...,1,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,"{'latitude': 55.853449, 'longitude': -4.199841}",[],NaN,"{'address1': 'Unit 55 Forge Shopping Centre', ...",+441412371282,+44 141 237 1282,1012.326576
2000,s0ZVlYq5euYHpQEC8LL8YQ,bbs-coffee-and-muffins-glasgow-2,BB's Coffee and Muffins,https://s3-media2.fl.yelpcdn.com/bphoto/5aBsWM...,False,https://www.yelp.com/biz/bbs-coffee-and-muffin...,1,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 55.854661, 'longitude': -4.208116}",[],NaN,"{'address1': 'The Forge Shopping Centre', 'add...",+441415547878,+44 141 554 7878,1012.326576


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [40]:
# Copy over the columns to analyse
selected_yelp_POIs_df = yelp_POIs_df[['name','categories','location','rating','review_count']].copy()
selected_yelp_POIs_df

,name,categories,location,rating,review_count
0,Café Gandolfi,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'address1': '64 Albion Street', 'address2': '...",4.5,67
1,Hutchesons Bar & Brasserie,"[{'alias': 'scottish', 'title': 'Scottish'}, {...","{'address1': '158 Ingram Street', 'address2': ...",4.0,51
2,BrewDog DogHouse,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...","{'address1': '99 Hutcheson Street', 'address2'...",4.5,48
3,The Wilson Street Pantry,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","{'address1': '6 Wilson Street', 'address2': ''...",4.5,88
4,Paesano Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'address1': '94 Miller Street', 'address2': '...",4.5,154
...,...,...,...,...,...
1997,Rab Haws Kitchens,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]","{'address1': '111 Helenvale Street', 'address2...",5.0,2
1998,The Bank,"[{'alias': 'pubs', 'title': 'Pubs'}]","{'address1': '1450 Gallowgate', 'address2': No...",3.0,1
1999,McChans,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'address1': 'Unit 55 Forge Shopping Centre', ...",3.0,1
2000,BB's Coffee and Muffins,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'address1': 'The Forge Shopping Centre', 'add...",3.0,1


In [41]:
# Gather Addresss and Catagory names using the previously created functions used on the Foursquare data

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_yelp_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address1', '')
    
    return poi_address

# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_yelp_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a tuple
    # NOTE: Tried list before, but .drop_duplicates() does not work on that data type so I had to settle with tuples
    category_tuple = ()
    for category in categories_dic:
        category_tuple += (category['title'],) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_tuple


selected_yelp_POIs_df['address'] = selected_yelp_POIs_df.apply(get_yelp_address, axis=1)
selected_yelp_POIs_df['category_names'] = selected_yelp_POIs_df.apply(get_yelp_category_names, axis=1)

selected_yelp_POIs_df

,name,categories,location,rating,review_count,address,category_names
0,Café Gandolfi,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'address1': '64 Albion Street', 'address2': '...",4.5,67,64 Albion Street,"(Seafood, British)"
1,Hutchesons Bar & Brasserie,"[{'alias': 'scottish', 'title': 'Scottish'}, {...","{'address1': '158 Ingram Street', 'address2': ...",4.0,51,158 Ingram Street,"(Scottish, Seafood, Steakhouses)"
2,BrewDog DogHouse,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...","{'address1': '99 Hutcheson Street', 'address2'...",4.5,48,99 Hutcheson Street,"(Pubs, Beer, Wine & Spirits, Barbeque)"
3,The Wilson Street Pantry,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","{'address1': '6 Wilson Street', 'address2': ''...",4.5,88,6 Wilson Street,"(Cafes, Coffee & Tea, Breakfast & Brunch)"
4,Paesano Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'address1': '94 Miller Street', 'address2': '...",4.5,154,94 Miller Street,"(Pizza, Italian)"
...,...,...,...,...,...,...,...
1997,Rab Haws Kitchens,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]","{'address1': '111 Helenvale Street', 'address2...",5.0,2,111 Helenvale Street,"(Fast Food,)"
1998,The Bank,"[{'alias': 'pubs', 'title': 'Pubs'}]","{'address1': '1450 Gallowgate', 'address2': No...",3.0,1,1450 Gallowgate,"(Pubs,)"
1999,McChans,"[{'alias': 'chinese', 'title': 'Chinese'}]","{'address1': 'Unit 55 Forge Shopping Centre', ...",3.0,1,Unit 55 Forge Shopping Centre,"(Chinese,)"
2000,BB's Coffee and Muffins,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'address1': 'The Forge Shopping Centre', 'add...",3.0,1,The Forge Shopping Centre,"(Coffee & Tea,)"


In [42]:
# Drop columns that we have already parsed
selected_yelp_POIs_df = selected_yelp_POIs_df.drop(['categories','location'], axis=1)

selected_yelp_POIs_df = selected_yelp_POIs_df.drop_duplicates()

# Final view before export to .csv
selected_yelp_POIs_df

,name,rating,review_count,address,category_names
0,Café Gandolfi,4.5,67,64 Albion Street,"(Seafood, British)"
1,Hutchesons Bar & Brasserie,4.0,51,158 Ingram Street,"(Scottish, Seafood, Steakhouses)"
2,BrewDog DogHouse,4.5,48,99 Hutcheson Street,"(Pubs, Beer, Wine & Spirits, Barbeque)"
3,The Wilson Street Pantry,4.5,88,6 Wilson Street,"(Cafes, Coffee & Tea, Breakfast & Brunch)"
4,Paesano Pizza,4.5,154,94 Miller Street,"(Pizza, Italian)"
...,...,...,...,...,...
1979,The Amethyst,4.0,3,1209 Govan Road,"(Pubs,)"
1980,Dowanhill Park,4.5,7,Havelock St/ Highburgh Rd,"(Playgrounds, Parks)"
1982,Roll With It,1.0,1,1145 Govan Road,"(Cafes, Delis, Soup)"
1983,Polka Dot Caffe,4.0,1,10 Holmfauldhead Place,"(Cafes,)"


In [43]:
# Export to .csv
selected_yelp_POIs_df.to_csv('../data/Yelp_POIs.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In my opinion the Yelp API provided more complete data. Not only did Yelp have an extra ~100 points of interest, but the quality and amount of data was much better (16 columns vs 10).

Get the top 10 restaurants according to their rating

In [44]:
# First thing I noticed was that a lot of these places with 5.0 ratings have very few numbers of reviews
selected_yelp_POIs_df.sort_values('rating', ascending=False).head(10)

,name,rating,review_count,address,category_names
143,Greggs,5.0,1,None,"(Bakeries, Sandwiches)"
1343,Abarcrombys Cafe Bar,5.0,2,180 Dumbarton Road,"(Bars, Cafes)"
1381,Enzos Fish & Chips,5.0,1,1200 Shettleston Road,"(Fish & Chips, Fast Food)"
1378,McDonald's Restaurants,5.0,1,10 Westhorn Drive,"(American (Traditional),)"
1800,Mcharg Bakers' Shops,5.0,1,1493 Paisley Road West,"(Bakeries,)"
701,Ecco 808 il Cafe,5.0,4,808 Crow Road,"(Italian, Butcher)"
1356,Magic Grill,5.0,1,1341 Dumbarton Road,"(Restaurants,)"
1354,Cupcakes For You & West End Vintage,5.0,1,1251 Dumbarton Road,"(Cupcakes,)"
421,Old Black Bull,5.0,1,1316-1318 Gallowgate,"(Pubs,)"
707,Beefcake Cafe,5.0,1,808 Crow Road,"(Cafes,)"


In [63]:
# To get more accurate results, lets limit our search to places with 10 or more reviews
top_glasgow_restaurants = selected_yelp_POIs_df[selected_yelp_POIs_df['review_count'] >= 10]

# Sort by rating
top_glasgow_restaurants = top_glasgow_restaurants.sort_values('rating', ascending=False)

# Limit to top 10
top_glasgow_restaurants.head(10)

,name,rating,review_count,address,category_names
397,River Kelvin Walkway,5.0,12,,"(Local Flavor, Landmarks & Historical Building..."
630,Obsession of India,5.0,13,25 High Street,"(Indian,)"
616,Picnic,5.0,10,103 Ingram Street,"(Cafes, Breakfast & Brunch, Desserts)"
517,Two Fat Ladies,5.0,14,88 Dumbarton Road,"(Seafood,)"
331,Milk Cafe,5.0,10,452 Victoria Road,"(Cafes, Community Service/Non-Profit)"
483,Wee Lochan,5.0,15,340 Crow Road,"(Scottish,)"
341,Black Sheep Bistro,5.0,14,10 Clarendon Street,"(British,)"
342,Yelp Event: Food Photography & Styling Masterc...,5.0,13,Ocho,"(Yelp Events,)"
320,Sapori d'Italia,5.0,16,1021 Cathcart Road,"(Italian,)"
343,Ka Ka Lok,4.5,23,175 Saint Georges Road,"(Chinese,)"


In [64]:
# We can see that some non restaurant items were included, because Yelp does not have a "restaurant" category we will have to filter out unwanted categories
values_to_remove = ["Yelp Events","Parks"]

# The following function looks through each tuple and will return true or false if the value is found or not
# used to filter out unwanted values
def check_value(tuple):
    found = False
    
    for value in values_to_remove:
        if value in tuple:
            found = True
            break
    
    return not found

# Apply the filter
top_glasgow_restaurants_filtered = top_glasgow_restaurants[top_glasgow_restaurants['category_names'].apply(check_value)]

top_glasgow_restaurants_filtered.head(10)

,name,rating,review_count,address,category_names
630,Obsession of India,5.0,13,25 High Street,"(Indian,)"
616,Picnic,5.0,10,103 Ingram Street,"(Cafes, Breakfast & Brunch, Desserts)"
517,Two Fat Ladies,5.0,14,88 Dumbarton Road,"(Seafood,)"
331,Milk Cafe,5.0,10,452 Victoria Road,"(Cafes, Community Service/Non-Profit)"
483,Wee Lochan,5.0,15,340 Crow Road,"(Scottish,)"
341,Black Sheep Bistro,5.0,14,10 Clarendon Street,"(British,)"
320,Sapori d'Italia,5.0,16,1021 Cathcart Road,"(Italian,)"
343,Ka Ka Lok,4.5,23,175 Saint Georges Road,"(Chinese,)"
322,Battlefield Rest,4.5,27,55 Battlefield Road,"(Italian, Brasseries)"
1334,Ardnamurchan,4.5,15,325 Hope Street,"(Scottish, Wine Bars)"
